In [1]:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from typing import List
from sqlite3 import connect



In [2]:
data: pd.DataFrame  = pd.read_csv('datasets/chap2/earthquakes.csv', parse_dates=True, )
data.head()

,alert,cdi,code,detail,dmin,felt,gap,ids,mag,magType,...,sources,status,time,title,tsunami,type,types,tz,updated,url
0,NaN,NaN,37389218,https://earthquake.usgs.gov/fdsnws/event/1/que...,0.008693,NaN,85.0,",ci37389218,",1.35,ml,...,",ci,",automatic,1539475168010,"M 1.4 - 9km NE of Aguanga, CA",0,earthquake,",geoserve,nearby-cities,origin,phase-data,",-480.0,1539475395144,https://earthquake.usgs.gov/earthquakes/eventp...
1,NaN,NaN,37389202,https://earthquake.usgs.gov/fdsnws/event/1/que...,0.020030,NaN,79.0,",ci37389202,",1.29,ml,...,",ci,",automatic,1539475129610,"M 1.3 - 9km NE of Aguanga, CA",0,earthquake,",geoserve,nearby-cities,origin,phase-data,",-480.0,1539475253925,https://earthquake.usgs.gov/earthquakes/eventp...
2,NaN,4.4,37389194,https://earthquake.usgs.gov/fdsnws/event/1/que...,0.021370,28.0,21.0,",ci37389194,",3.42,ml,...,",ci,",automatic,1539475062610,"M 3.4 - 8km NE of Aguanga, CA",0,earthquake,",dyfi,focal-mechanism,geoserve,nearby-cities,o...",-480.0,1539536756176,https://earthquake.usgs.gov/earthquakes/eventp...
3,NaN,NaN,37389186,https://earthquake.usgs.gov/fdsnws/event/1/que...,0.026180,NaN,39.0,",ci37389186,",0.44,ml,...,",ci,",automatic,1539474978070,"M 0.4 - 9km NE of Aguanga, CA",0,earthquake,",geoserve,nearby-cities,origin,phase-data,",-480.0,1539475196167,https://earthquake.usgs.gov/earthquakes/eventp...
4,NaN,NaN,73096941,https://earthquake.usgs.gov/fdsnws/event/1/que...,0.077990,NaN,192.0,",nc73096941,",2.16,md,...,",nc,",automatic,1539474716050,"M 2.2 - 10km NW of Avenal, CA",0,earthquake,",geoserve,nearby-cities,origin,phase-data,scit...",-480.0,1539477547926,https://earthquake.usgs.gov/earthquakes/eventp...


In [3]:
# converting to CSV file with no index
data.to_csv("output.csv", index=False)

In [4]:
import sqlite3
with sqlite3.connect('datasets/quakes.db') as connection:
    # Creates a connection. Makes a new database
    pd.read_csv('datasets/chap2/tsunamis.csv').to_sql("tsunami", connection, index=False, if_exists='replace')
    # Read the csv file tsunami and makes it to a sql database. If the sql name already exist then replace it.
connection.close()

with sqlite3.connect('datasets/quakes.db') as connection:
    tsunami = pd.read_sql('SELECT * from tsunami', connection)
tsunami.dtypes

alert       object
type        object
title       object
place       object
magType     object
mag        float64
time         int64
dtype: object

Pandas indexing
We can use loc or iloc
We frst provide the row and then the column index
Example:
    data[row index, colum index]

In [5]:
data.loc[100:103,'title']



100        M 1.2 - 25km NW of Ester, Alaska
101    M 0.6 - 8km ESE of Mammoth Lakes, CA
102          M 1.3 - 8km ENE of Aguanga, CA
103             M 0.0 - 5km WSW of Anza, CA
Name: title, dtype: object

In [6]:
# We can select multiple rows
data.loc[10:15, ["title", "mag"]]

,title,mag
10,"M 0.5 - 10km NE of Aguanga, CA",0.50
11,"M 2.8 - 53km SE of Punta Cana, Dominican Republic",2.77
12,"M 0.5 - 9km NE of Aguanga, CA",0.50
13,"M 4.5 - 120km SSW of Banda Aceh, Indonesia",4.50
14,"M 2.1 - 14km NW of Parkfield, CA",2.13
15,"M 2.0 - 156km WNW of Haines Junction, Canada",2.00


Filtering Data


In [7]:
# Select data that its greater than 7
data[data.mag >= 7.0]
# But we can also use loc
# Select from the rows that mag greater than 7 but only select columns that that we specify
data.loc[data.mag >= 7.0 , ['alert', 'mag', 'magType', 'title', 'tsunami', 'type']]


,alert,mag,magType,title,tsunami,type
837,green,7.0,mww,"M 7.0 - 117km E of Kimbe, Papua New Guinea",1,earthquake
5263,red,7.5,mww,"M 7.5 - 78km N of Palu, Indonesia",1,earthquake


In [8]:
# example using logical operators inside loc
data.loc[(data.tsunami == 1) & (data.alert == "red"  ), ['alert', 'mag', 'magType', 'title', 'tsunami', 'type']]

,alert,mag,magType,title,tsunami,type
5263,red,7.5,mww,"M 7.5 - 78km N of Palu, Indonesia",1,earthquake


In [9]:
# selecting data from a column with strings and has not null
data.loc[data.place.str.contains("Alaska") & (data.alert.notnull()) , 
        ['alert', 'mag', 'magType', 'title', 'tsunami', 'type'] ]

,alert,mag,magType,title,tsunami,type
1015,green,5.0,ml,"M 5.0 - 61km SSW of Chignik Lake, Alaska",1,earthquake
1273,green,4.0,ml,"M 4.0 - 71km SW of Kaktovik, Alaska",1,earthquake
1795,green,4.0,ml,"M 4.0 - 60km WNW of Valdez, Alaska",1,earthquake
2752,green,4.0,ml,"M 4.0 - 67km SSW of Kaktovik, Alaska",1,earthquake
3260,green,3.9,ml,"M 3.9 - 44km N of North Nenana, Alaska",0,earthquake
4101,green,4.2,ml,"M 4.2 - 131km NNW of Arctic Village, Alaska",0,earthquake
6897,green,3.8,ml,"M 3.8 - 80km SSW of Kaktovik, Alaska",0,earthquake
8524,green,3.8,ml,"M 3.8 - 69km SSW of Kaktovik, Alaska",0,earthquake
9133,green,5.1,ml,"M 5.1 - 64km SSW of Kaktovik, Alaska",1,earthquake


Regular Expression

In [10]:
data.loc[(data.place.str.contains(r'CA|California$')) &
        (data.mag > 3.8) , 
        ['alert', 'mag', 'magType', 'title', 'tsunami', 'type']]

,alert,mag,magType,title,tsunami,type
1465,green,3.83,mw,"M 3.8 - 109km WNW of Trinidad, CA",0,earthquake
2414,green,3.83,mw,"M 3.8 - 5km SW of Tres Pinos, CA",1,earthquake


In [14]:
# Choosing in Between Values
data.loc[data.magType.isin(["mw", "mwb"]), ["alert" , "mag" , "magType"]]

,alert,mag,magType
995,NaN,3.35,mw
1465,green,3.83,mw
2414,green,3.83,mw
4988,green,4.41,mw
6307,green,5.80,mwb
8257,green,5.70,mwb


In [15]:
# grabbing the lowest and highest indexes\
data.loc[[data.mag.idxmin(), data.mag.idxmax()] , ['alert', 'mag', 'magType', 'title', 'tsunami', 'type']
]

,alert,mag,magType,title,tsunami,type
2409,NaN,-1.26,ml,"M -1.3 - 41km ENE of Adak, Alaska",0,earthquake
5263,red,7.50,mww,"M 7.5 - 78km N of Palu, Indonesia",1,earthquake


In [18]:
# Adding or removing data
df = data.copy()
df["mag_negative"] = df.mag < 0
df["mag_negative"]

0       False
1       False
2       False
3       False
4       False
        ...  
9327    False
9328    False
9329    False
9330    False
9331    False
Name: mag_negative, Length: 9332, dtype: bool

In [22]:
# dropping Columns
cols_to_drop = [col for col in df.columns 
    if col not in [
    'alert', 'mag', 'title', 'time', 'tsunami']
    ]

df.drop(columns=cols_to_drop , inplace=False)

,alert,mag,time,title,tsunami
0,NaN,1.35,1539475168010,"M 1.4 - 9km NE of Aguanga, CA",0
1,NaN,1.29,1539475129610,"M 1.3 - 9km NE of Aguanga, CA",0
2,NaN,3.42,1539475062610,"M 3.4 - 8km NE of Aguanga, CA",0
3,NaN,0.44,1539474978070,"M 0.4 - 9km NE of Aguanga, CA",0
4,NaN,2.16,1539474716050,"M 2.2 - 10km NW of Avenal, CA",0
...,...,...,...,...,...
9327,NaN,0.62,1537230228060,"M 0.6 - 9km ENE of Mammoth Lakes, CA",0
9328,NaN,1.00,1537230135130,"M 1.0 - 3km W of Julian, CA",0
9329,NaN,2.40,1537229908180,"M 2.4 - 35km NNE of Hatillo, Puerto Rico",0
9330,NaN,1.10,1537229545350,"M 1.1 - 9km NE of Aguanga, CA",0
